<a href="https://colab.research.google.com/github/LiZhen97/Deep-Neural-Networks/blob/main/assignment_class4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 4 Assignment: Classification and Regression Neural Network**

**Student Name: Your Name**

# Assignment Instructions

For this assignment, you will use the **crx.csv** dataset.  This dataset is a public dataset that can you can find [here](https://archive.ics.uci.edu/ml/datasets/credit+approval). You should use the CSV file on my data site, at this location: [crx.csv](https://data.heatonresearch.com/data/t81-558/crx.csv) because it includes column headers.  The primary use for this dataset is binary classification. There are 15 attributes, plus a target column that contains only + or -.  Some of the columns have missing values.

You should train a neural network and return the predictions.  You will submit these predictions to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Complete the following tasks:

* Your task is to replace missing values in columns *a2* and *a14* with values estimated by a neural network (one neural network for *a2* and another for *a14*).
* Your submission file will contain the same headers as the source CSV: *a1*, *a2*, *s3*, *a4*, *a5*, *a6*, *a7*, *a8*, *a9*, *a10*, *a11*, *a12*, *a13*, *a14*, *a15*, and *a16*.
* You should only need to modify *a2* and *a14*.
* Neural networks can be much more powerful at filling missing variables than median and mean.
* Train two neural networks to predict *a2* and *a14*.  
* The *y* (target) for training the two nets will be *a2* and *a14*, depending on which you are trying to fill.
* The x for training the two nets will be 's3','a8','a9','a10','a11','a12','a13','a15'.  These are chosen because it is important not to use any columns with missing values; also, it could cause unwanted bias if we include the ultimate target (*a16*).
* ONLY predict new values for missing values in *a2* and *a14*.
* You will likely get this small warning:  Warning: The mean of column a14 differs from the solution file by 0.20238937709643778. (might not matter if small)



# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


# Assignment Submit Function

You will submit the ten programming assignments electronically.  The following **submit** function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any underlying problems. 

**It is unlikely that should need to modify this function.**

In [3]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #4 Sample Code

The following code provides a starting point for this assignment.

In [23]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

# This is your student key that I emailed to you at the beginnning of the semester.
key = "eaT4qNtOTV4YNWzOIPMr83juoEsVZAkH7gPcxQ4P"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/My Drive/Colab Notebooks/assignment_ZhenLi_class4.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class4.ipynb'  # Windows
#file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class4.ipynb'  # Mac/Linux

# Begin assignment
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/crx.csv",na_values=['?'])

In [5]:
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics

In [6]:
df

,a1,a2,s3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-


In [7]:
# encode the features
# use s3','a8','a9','a10','a11','a12','a13','a15 to be X
x = df[['s3','a8','a9','a10','a11','a12','a13','a15','a2','a14']].copy()
x = pd.concat([x,pd.get_dummies(df['a9'],prefix="a9")],axis=1)
x.drop('a9', axis=1, inplace=True)

x = pd.concat([x,pd.get_dummies(df['a10'],prefix="a10")],axis=1)
x.drop('a10', axis=1, inplace=True)

x = pd.concat([x,pd.get_dummies(df['a12'],prefix="a12")],axis=1)
x.drop('a12', axis=1, inplace=True)

x = pd.concat([x,pd.get_dummies(df['a13'],prefix="a13")],axis=1)
x.drop('a13', axis=1, inplace=True)


In [14]:
x

,s3,a8,a11,a15,a2,a14,a9_f,a9_t,a10_f,a10_t,a12_f,a12_t,a13_g,a13_p,a13_s
0,0.000,1.25,1,0,30.83,202.0,0,1,0,1,1,0,1,0,0
1,4.460,3.04,6,560,58.67,43.0,0,1,0,1,1,0,1,0,0
2,0.500,1.50,0,824,24.50,280.0,0,1,1,0,1,0,1,0,0
3,1.540,3.75,5,3,27.83,100.0,0,1,0,1,0,1,1,0,0
4,5.625,1.71,0,0,20.17,120.0,0,1,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,10.085,1.25,0,0,21.08,260.0,1,0,1,0,1,0,1,0,0
686,0.750,2.00,2,394,22.67,200.0,1,0,0,1,0,1,1,0,0
687,13.500,2.00,1,1,25.25,200.0,1,0,0,1,0,1,1,0,0
688,0.205,0.04,0,750,17.92,280.0,1,0,1,0,1,0,1,0,0


In [26]:
from re import X
from scipy.stats import zscore
# replace missing values in columns a2 and a14 with values estimated by a neural network (one neural network for a2 and another for a14)
# use s3','a8','a9','a10','a11','a12','a13','a15 to be X
def fill_missing_numeric(df,target):
    # Fill in as needed
    new_df = df.dropna()
    print(new_df)
    x = new_df.drop(['a2', 'a14'], axis=1)
    x = x.values
    y = new_df[target].values
    #split dataset
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=11)
    # build neural network
    # Build the neural network
    model = Sequential()
    model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dense(10, activation='relu')) # Hidden 2
    model.add(Dense(1)) # Output
    model.compile(loss='mean_squared_error', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto', 
                        restore_best_weights=True)
    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)
    
    # Predict
    pred = model.predict(x_test)

    # Measure MSE error.  
    score = metrics.mean_squared_error(pred,y_test)
    print("Final score (MSE): {}".format(score))

    #caculate missing value
    print("calculate missing value")
    missing_rows = df.loc[df[target].isnull()]
    print(missing_rows)
    for i in range(len(df)):
      if np.isnan(df.loc[i, target]):
        row = df.iloc[i, :].drop(['a2','a14'])
        #row = row.values
       # print("----")
        y_pred = model.predict(row.values.reshape(1, -1))
        df.loc[i, target] = y_pred[0][0]
       # df[i, 'a'] = y_pred
    return df

res = fill_missing_numeric(x, 'a2')
res = fill_missing_numeric(x, 'a14')



           s3    a8  a11       a15     a2    a14  a9_f  a9_t  a10_f  a10_t  \
0   -0.956613  1.25    1 -0.195413  30.83  202.0     0     1      0      1   
1   -0.060051  3.04    6 -0.087852  58.67   43.0     0     1      0      1   
2   -0.856102  1.50    0 -0.037144  24.50  280.0     0     1      1      0   
3   -0.647038  3.75    5 -0.194837  27.83  100.0     0     1      0      1   
4    0.174141  1.71    0 -0.195413  20.17  120.0     0     1      1      0   
..        ...   ...  ...       ...    ...    ...   ...   ...    ...    ...   
685  1.070704  1.25    0 -0.195413  21.08  260.0     1     0      1      0   
686 -0.805846  2.00    2 -0.119736  22.67  200.0     1     0      0      1   
687  1.757198  2.00    1 -0.195221  25.25  200.0     1     0      0      1   
688 -0.915403  0.04    0 -0.051358  17.92  280.0     1     0      1      0   
689 -0.278161  8.29    0 -0.195413  35.00    0.0     1     0      1      0   

     a12_f  a12_t  a13_g  a13_p  a13_s  
0        1      0     

In [32]:
df

,a1,a2,s3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-


In [33]:
df['a2'] = res['a2']
df['a14'] = res['a14']
submit(source_file=file,data=[df],key=key,no=4)

Success: Submitted Assignment 4 for li.zhen1:
You have submitted this assignment 2 times. (this is fine)
Note: The mean difference 0.012322538010138828 for column 'a2' is acceptable and is less than the maximum allowed value of '1.0' for this assignment.
Note: The mean difference 0.567602926060772 for column 'a14' is acceptable and is less than the maximum allowed value of '1.0' for this assignment.
No warnings or errors (only notes), you will probably do well, but no guarantee. :-)
